# Автоматическое распознавание новообразований в "русском английском" текстов разных жанров #
## Анастасия Буракова, БКЛ 181 ##

**Описание:** программа распознает в текстовых документах новообразования из [собранного корпуса](https://github.com/nstsi/find_neologisms/blob/main/data/neologisms_table.csv).

In [ ]:
import re
import pandas as pd

import nltk

nltk.download('punkt')

### 1. Сбор данных

Чтобы **проверить программу на тренировочном файле** DAr_8_1.txt, запустите
код ниже и
пропустите следующую ячейку:

In [ ]:
DATA_FOLDER = 'data/'

FILE_TO_PROCESS = 'DAr_8_1.txt'
FILE_TO_PROCESS_PATH: str = f'{DATA_FOLDER}{FILE_TO_PROCESS}'

Чтобы программа обработала **ваш текст** на английском языке, переведите его в
формат .txt и положите в папку data, находящуюся в одной папке с этой программой. Далее запустите код ниже и введите имя вашего файла в формате filename.txt:

In [ ]:
DATA_FOLDER = 'data/'
FILE_TO_PROCESS = input()
FILE_TO_PROCESS_PATH: str = f'{DATA_FOLDER}{FILE_TO_PROCESS}'
check_format = re.findall(r'.+\.txt', FILE_TO_PROCESS)
while not os.path.exists(FILE_TO_PROCESS_PATH) or not check_format:
    print(f'Положите файл в формате .txt в папку data, находящуюся в одной'
          f'папке с этой программой, и введите имя вашего файла'
          f'в формате filename.txt.')
    FILE_TO_PROCESS = input()
FILE_TO_PROCESS_PATH: str = f'{DATA_FOLDER}{FILE_TO_PROCESS}'
check_format = re.findall(r'.+\.txt', FILE_TO_PROCESS)

*Запуск **всех** следующих ячеек **обязателен** для исправной работы
программы.*

In [ ]:
FILE_TO_PROCESS_NAME = FILE_TO_PROCESS[:-4]
RESULTS_FILE: str = f'{FILE_TO_PROCESS_NAME}_results.txt'

### 2. Подготовка данных
Сначала прочитаем и очистим рассматриваемый текст, а также разделим его на предложения:

In [ ]:
def read_text(filename: str) -> str:
    with open(filename, encoding='utf-8') as txt:
        text_ = txt.read()
    return text_


def make_one_line(text_: str) -> str:
    text_ = re.sub('\n', ' ', text_)
    return text_


def split_in_sentences(text_: str) -> list:
    sentences_ = nltk.tokenize.sent_tokenize(text_)
    return sentences_

Теперь создадим из корпуса новообразований датафрейм: 

In [ ]:
def make_neologisms_df():
    neologisms_ = pd.read_csv('data/neologisms_table.csv', delimiter=',')
    return neologisms_

### 3. Обработка данных
Теперь проверим текст на наличие каждого новообразования и добавим в новый датафрейм параметры каждого найденного новообразования, первое предложение, в котором новообразование встретилось в данном тексте, а также предшествующее и следующее предложений (контекст).


In [ ]:
def find_neologisms(text_one_line_: str,
                    sentences_: list,
                    neologisms_):

    # Названия колонок берем из существующего дф и добавляем еще 3 колонки:
    column_names = neologisms_.columns.tolist()
    columns_add = ['current_sentence', 'previous_sentence', 'next_sentence']
    for elem in columns_add:
        column_names.append(elem)

    # Создаем датафрейм с этими колонками:
    results = pd.DataFrame(columns=column_names)

    # Идем по строкам дф с новообразованиями и ищем их в тексте:
    for index, row in neologisms_.iterrows():

    # Если находим, сохраняем контекст:
        if row['word'] in text_one_line_:
            for i, sentence in enumerate(sentences_):

    # Текущее предложение есть всегда:
                if row['word'] in sentence:
                    current_sentence = sentence

    # А вот предыдушего или следующего может не быть:
                    try:
                        previous_sentence = sentences_[i-1]
                    except IndexError:
                        previous_sentence = 'пусто'

                    try:
                        next_sentence = sentences_[i+1]
                    except IndexError:
                        next_sentence = 'пусто'

    # Записываем все параметры новообразования:
                    res_list = [row['word'],
                                row['meaning'],
                                row['linguistic process'],
                                row['mistake location'],
                                row['mistake type'],
                                row['L1 interference'],
                                row['source'],
                                row['enTenTen18'],
                                current_sentence,
                                previous_sentence,
                                next_sentence]

    # И добавляем их к датафрейму с результатами:
                    df_length = len(results)
                    results.loc[df_length] = res_list
                    break  # Берём только первое вхождение

    return results

### 4. Запись результатов
Если ни одно новообразование не обнаружено в тексте, эта функция выведет в консоль сообщение типа: «В тексте DAr_8_1.txt не были найдены новообразования».

Если образования были обнаружены, эта функция создаст файл результатов и запишет туда все полученные данные.



In [ ]:
def save_to_file(results, results_filename, text_filename):

    if results.empty:
        print(f'В тексте {text_filename} не были найдены новообразования.')

    else:
        with open(results_filename, 'w', encoding='utf-8') as f:

            f.write(f'В тексте {text_filename} были найдены следующие '
                    f'новообразования:')
            f.write('\n')

            for i in range(results.shape[0]):

                f.write(f'{i+1}.')
                f.write('\n')

                word = results.iloc[i]['word']
                meaning = results.iloc[i]['meaning']
                ling_proc = results.iloc[i]['linguistic process']
                mistake_loc = results.iloc[i]['mistake location']
                print(mistake_loc)
                mistake_type = results.iloc[i]['mistake type']
                print(mistake_type)
                l1_interference = results.iloc[i]['L1 interference']
                en18 = results.iloc[i]['enTenTen18']

                f.write(f'• Новообразование: {word} | '
                        f'Значение: {meaning} | '
                        f'Лингвистический процесс: {ling_proc} | '
                        f'Локация ошибки: {mistake_loc} | '
                        f'Тип ошибки: {mistake_type} | '
                        f'Возможность L1 interference: {l1_interference} | '
                        f'Количество вхождений в enTenTen18: {en18}')
                f.write('\n')

                curr_sent = results.iloc[i]['current_sentence']
                f.write(f'• Предложение: {curr_sent}')
                f.write('\n')

                prev_sent = results.iloc[i]['previous_sentence']
                next_sent = results.iloc[i]['next_sentence']
                context = f'{prev_sent} {curr_sent} {next_sent}'
                clean_context = re.sub(' пусто', '', context)

                f.write(f'• Контекст: {clean_context}')
                f.write('\n')

                source = results.iloc[i]['source']
                f.write(f'• Другие тексты с этим новообразованием: {source}')
                f.write('\n')

                if results.iloc[i][0] == results.iloc[-1][0]:
                    print(f'Найденные новообразования записаны в файл '
                          f'{results_filename}.')

### 5. Объединяем всё вместе:

In [ ]:
# Читаем текст:
text = read_text(FILE_TO_PROCESS_PATH)

# Записываем текст одной строкой:
text_one_line = make_one_line(text)

# Делим текст на предложения:
sentences = split_in_sentences(text_one_line)

# Создаем из корпуса новообразований датафрейм:
neologisms = make_neologisms_df()

# Ищем новообразования:
results_df = find_neologisms(text_one_line, sentences, neologisms)

# Записываем результаты в файл и выводим в консоль итог работы программы
save_to_file(results_df, RESULTS_FILE, FILE_TO_PROCESS_NAME)